### 1. Imports

In [45]:
import requests
import json
import pandas as pd

In [42]:
uri = 'https://api.football-data.org/v4/competitions/BL1/matches?season=2020'
headers = {'X-Auth-Token': '941e5424e9374e28817b075859d2867c'}

In [43]:
response = requests.get(uri, headers=headers)

In [44]:
response.json()

{'filters': {'season': 2020},
 'resultSet': {'count': 306,
  'first': '2020-09-18',
  'last': '2021-05-22',
  'played': 306},
 'competition': {'id': 2002,
  'name': 'Bundesliga',
  'code': 'BL1',
  'type': 'LEAGUE',
  'emblem': 'https://crests.football-data.org/BL1.png'},
 'matches': [{'area': {'id': 2088,
    'name': 'Germany',
    'code': 'DEU',
    'flag': 'https://crests.football-data.org/759.svg'},
   'competition': {'id': 2002,
    'name': 'Bundesliga',
    'code': 'BL1',
    'type': 'LEAGUE',
    'emblem': 'https://crests.football-data.org/BL1.png'},
   'season': {'id': 599,
    'startDate': '2020-09-18',
    'endDate': '2021-05-22',
    'currentMatchday': 34,
    'winner': {'id': 5,
     'name': 'FC Bayern München',
     'shortName': 'Bayern',
     'tla': 'FCB',
     'crest': 'https://crests.football-data.org/5.svg',
     'address': 'Säbenerstr. 51 München 81547',
     'website': 'http://www.fcbayern.de',
     'founded': 1900,
     'clubColors': 'Red / White / Blue',
     'venu

In [ ]:
df = pdf